In [18]:
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
import cvxpy as cp
import numpy as np
import pandas as pd
import torch

In [19]:
df = pd.read_csv("stock_prices.csv").drop("date", axis=1)

In [20]:
sector_mapper = {
        "GOOG": "tech",
        "AAPL": "tech",
        "FB": "tech",
        "AMZN": "tech",
        "BABA": "tech",
        "GE": "utility",
        "AMD": "tech",
        "WMT": "retail",
        "BAC": "fig",
        "GM": "auto",
        "T": "auto",
        "UAA": "airline",
        "SHLD": "retail",
        "XOM": "energy",
        "RRC": "energy",
        "BBY": "retail",
        "MA": "fig",
        "PFE": "pharma",
        "JPM": "fig",
        "SBUX": "retail",
    }


In [21]:
mu = mean_historical_return(df)
S = CovarianceShrinkage(df).ledoit_wolf()

In [22]:

ds = []
for i in range(1000):
	max_tech = np.random.rand()
	max_retail = np.random.rand()
	max_fig = np.random.rand()
	max_energy = np.random.rand()
	utility_all = 0.2*np.random.rand()
	airline_all = 0.2*np.random.rand()
	rho = (10 - 1)*np.random.rand() + 1
	
	weights = cp.Variable(len(mu))
	constraints = [
		weights >= 0.0,
		cp.sum(weights) == 1.0,
		weights[5] == utility_all,
		weights[11] == airline_all,
		weights[[0,1,2,3,4,6]] <= max_tech,
		weights[[7,12,15,19]] <= max_retail,
		weights[[8,16,18]] <= max_fig,
		weights[[13,14]] <= max_energy
	]

	prob = cp.Problem(cp.Minimize(-(weights.T @ mu - 0.5 * rho* cp.quad_form(weights, S))),constraints)
	prob.solve()
	if prob.status == 'optimal': 
		X = np.array([max_tech, max_retail, max_fig, max_energy, utility_all, airline_all, rho])
		#y = np.array([w for w in list(weights.value)])
		ds.append((X, weights.value))

In [23]:
[ds[i][1] for i in range(len(ds))]

[array([ 1.27762362e-22,  1.42917280e-02,  1.80759013e-01,  5.13319478e-02,
         5.20773893e-02,  1.17054864e-02,  2.83235216e-23, -8.70706795e-23,
        -2.93740216e-23,  3.27057495e-23,  1.32097678e-22,  1.77381101e-01,
         7.85573273e-23,  2.39619371e-23,  6.07040237e-24,  2.10633059e-23,
         5.12453335e-01, -2.65882718e-23,  5.15510475e-24,  7.57286741e-23]),
 array([4.85400976e-22, 2.60491998e-22, 4.23113108e-22, 2.55722889e-22,
        6.26518311e-03, 2.80133823e-02, 1.20701721e-22, 2.02348950e-22,
        2.14065417e-22, 3.13199203e-22, 3.15285937e-22, 1.85090954e-01,
        4.29212246e-22, 3.16546152e-22, 1.94341293e-22, 2.73206406e-22,
        7.80630480e-01, 9.33123640e-23, 2.10745357e-22, 5.39433757e-22]),
 array([ 9.04896327e-02,  8.27622430e-02,  9.04896327e-02,  6.69483164e-02,
         9.04896327e-02,  1.72204959e-01, -1.70980279e-23,  1.84064847e-22,
         1.47139109e-22,  3.95339768e-23,  1.31182552e-22,  8.17451855e-02,
        -1.96823884e-23,  2.

In [24]:
ds[0][1][[0,1,2,3,4,6]].sum()

0.298460078295098

In [25]:
from pickle import dump
dump(ds, open("portfolio.pkl", "wb"))